In [ ]:
!pip install copernicusmarine

In [ ]:
from pprint import pprint
import xarray as xr
import copernicusmarine as copernicusmarine
print(copernicusmarine.__version__)
#only for the first time: 
#copernicusmarine.login() 

In [ ]:
catalogue = copernicusmarine.describe(
    include_datasets=True,
#    contains = ['European North West Shelf - Ocean Physics '],
#     contains = ['Atlantic-Iberian Biscay Irish- Ocean Physics']   
#    contains = ['thetao']
#
    contains=['Ocean Physics']
)
for index, product in enumerate(catalogue['products']):
    print(index, product['title'], product['product_id'])
    
    for index2, dataset in enumerate(product['datasets']):
       # if 'dataset_id' in dataset and 'H' in dataset['dataset_id']:        
        print(index,index2, 'dataset_id',dataset['dataset_id'], dataset['dataset_name'])

In [ ]:
catalogue['products'][0]['title']

In [ ]:
name='cmems_mod_ibi_phy_anfc_0.027deg'
name='cmems_mod_nws_phy_anfc_0.027deg'

catalogue = copernicusmarine.describe(
    include_datasets=True,
    contains = [name]
)

In [ ]:
uris_by_key = {}  # Dictionary to store URIs by key
def get_copernicus_data(name,format='arco-geo-series' ):
    #format='arco-geo-series' or 'arco-time-series'
    catalogue = copernicusmarine.describe(
    include_datasets=True,
    contains = [name],
    )
    for value in catalogue['products'][0]['datasets']:
        dataset_id = value['dataset_id']
        if any(substring in dataset_id for substring in ['static', '2D_PT1H-m', '3D_PT1H-m']):
            uris = []
            for service in value['versions'][0]['parts'][0]['services']:
                service_name = service['service_type']['service_name']
                if service_name in [format, 'static-arco']:
                    uris.append(service['uri'])
            uris_by_key[dataset_id] = uris
#    bbox = {"latitude": [48, 49], "longitude": [-6, -4]} 
    #.sel(latitude=slice(*bbox["latitude"]), longitude=slice(*bbox["longitude"]))
    thetao=(
        xr.open_dataset(uris_by_key[name+'-3D_PT1H-m'][0],
        engine='zarr',
        chunks={})[['thetao']]
    #.sel(latitude=slice(*bbox["latitude"]), longitude=slice(*bbox["longitude"]))
    )
    zos=(
        xr.open_dataset(uris_by_key[name+'-2D_PT1H-m'][0],
        engine='zarr',
        chunks={}).zos
         #.sel(latitude=slice(*bbox["latitude"]), longitude=slice(*bbox["longitude"]))
        )
    deptho=(
        xr.open_dataset(uris_by_key[name+'-3D_static'][0],
        engine='zarr',
        chunks={}).deptho
        #.sel(latitude=slice(*bbox["latitude"]), longitude=slice(*bbox["longitude"]))
        )
    mask=deptho.isnull()
    print('zos min,max value',zos.shape,zos.latitude[0:2].data,zos.latitude[-3:-1].data)
    print('deptho min,max value',deptho.shape,deptho.latitude[0:2].data,deptho.latitude[-3:-1].data)
    print('thetao min,max value',thetao.thetao.shape,thetao.latitude[0:2].data,thetao.latitude[-3:-1].data)
    ds=(
        thetao.rename({"thetao": "TEMP"})
        .assign(
                {
                    "XE": zos,
                    "H0": (deptho
                    ),
                    "mask": (mask
                    ),
                }
        )).rename({"latitude": "lat", "longitude": "lon", "elevation": "depth"})
    return ds
    

## name='cmems_mod_ibi_phy_anfc_0.027deg'
check if not mix with u,v,t coordinate on zarr?
compare with netcdf original file lat lon coordinate? 
only available from 2023-03-30, which does not corresopnds to the description.
check for both zarr format the time availavility 

In [ ]:
get_copernicus_data(name='cmems_mod_ibi_phy_anfc_0.027deg' )

## name='cmems_mod_nws_phy_anfc_0.027deg'
check if not mix with u,v,t coordinate on zarr?
compare with netcdf original file lat lon coordinate? 
only available from 2023-07-02, which does not corresopnds to the description.
check for both zarr format the time availavility 
verify if the data resolution is exactry same as what we have on datarmor


In [ ]:
get_copernicus_data(name='cmems_mod_nws_phy_anfc_0.027deg' )
